In [1]:
!pip install torch transformers sentence-transformers torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing inst

# Import libraries

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import seaborn as sns

# Dataset Class

In [3]:
class TextToGraphDataset(Dataset):
    def __init__(self, text_embeddings, graph_embeddings):
        self.text_embeddings = torch.tensor(text_embeddings, dtype=torch.float32)
        self.graph_embeddings = torch.tensor(graph_embeddings, dtype=torch.float32)

    def __len__(self):
        return len(self.text_embeddings)

    def __getitem__(self, idx):
        return self.text_embeddings[idx], self.graph_embeddings[idx]

# Mapping Model

In [4]:
class MappingTransformer(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=1024, output_dim=128, depth=6, dropout=0.3):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.layers = nn.ModuleList([
            nn.Sequential(
                nn.LayerNorm(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_dim, hidden_dim)
            ) for _ in range(depth)
        ])
        self.output_layer = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.input_layer(x)
        for layer in self.layers:
            residual = x
            x = layer(x)
            x += residual
        return self.output_layer(x)

class MultiInputFusionModel(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=1024, output_dim=128, depth=6, dropout=0.3):
        super().__init__()
        self.q_mapper = MappingTransformer(input_dim, hidden_dim, output_dim, depth, dropout)
        self.a_mapper = MappingTransformer(input_dim, hidden_dim, output_dim, depth, dropout)
        self.fusion_layer = nn.Sequential(
            nn.Linear(output_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, q, a):
        q_out = self.q_mapper(q)
        a_out = self.a_mapper(a)
        combined = torch.cat([q_out, a_out], dim=1)
        return self.fusion_layer(combined)


# Compute text embedding 

In [5]:
def compute_text_embeddings(text_list, encoder, tokenizer, device, batch_size=64):
    all_embeds = []
    encoder.eval()
    with torch.no_grad():
        for i in range(0, len(text_list), batch_size):
            batch_texts = text_list[i:i+batch_size]
            encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)
            outputs = encoder(**encoded_input, return_dict=True)
            # Lấy embedding CLS token, giả sử model trả về dạng last_hidden_state
            cls_embeds = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            all_embeds.append(cls_embeds)
    return np.vstack(all_embeds)

# -----------------------
# Normalize vectors
# -----------------------
def normalize(vectors):
    vectors = np.array(vectors)
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / np.clip(norms, 1e-8, None)

# -----------------------
# Dataset
# -----------------------
class DualInputDataset(Dataset):
    def __init__(self, q_vecs, a_vecs, g_vecs):
        self.q = torch.tensor(q_vecs, dtype=torch.float32)
        self.a = torch.tensor(a_vecs, dtype=torch.float32)
        self.g = torch.tensor(g_vecs, dtype=torch.float32)
    def __len__(self):
        return len(self.q)
    def __getitem__(self, idx):
        return self.q[idx], self.a[idx], self.g[idx]

# Training Function

In [6]:
def train_mapping_model(
    csv_path="/kaggle/input/question-and-abstract/combined_with_abstract.csv",
    model_name="intfloat/e5-large-v2",
    output_model_path="best_fusion_model.pt",
    epochs=300,
    batch_size=64,
    device=None,
    early_stop_patience=500,
):
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    print(f"Using device: {device}")

    print(f"📥 Loading encoder: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    encoder = AutoModel.from_pretrained(model_name).to(device)

    print("📂 Reading CSV...")
    df = pd.read_csv(csv_path)
    q_list, a_list, g_list = [], [], []

    for _, row in df.iterrows():
        questions = str(row["questions"]).split(";")
        abstract = str(row["abstract"])
        gvec = [float(row[str(i)]) for i in range(128)]

        for q in questions:
            q_list.append(q.strip())
            a_list.append(abstract)
            g_list.append(gvec)

    print(f"🧠 Computing embeddings for {len(q_list)} questions and abstracts...")
    q_embeds = compute_text_embeddings(q_list, encoder, tokenizer, device)
    a_embeds = compute_text_embeddings(a_list, encoder, tokenizer, device)

    print("🔄 Normalizing embeddings...")
    q_embeds = normalize(q_embeds)
    a_embeds = normalize(a_embeds)
    g_list = normalize(np.array(g_list))

    print("📊 Splitting train/test sets by samples...")
    q_train, q_test, a_train, a_test, g_train, g_test = train_test_split(
        q_embeds, a_embeds, g_list, test_size=0.1, random_state=42
    )

    class DualInputDataset(Dataset):
        def __init__(self, qs, abs, gs):
            self.qs = torch.tensor(qs, dtype=torch.float32)
            self.abs = torch.tensor(abs, dtype=torch.float32)
            self.gs = torch.tensor(gs, dtype=torch.float32)
        def __len__(self): return len(self.qs)
        def __getitem__(self, idx): return self.qs[idx], self.abs[idx], self.gs[idx]

    train_loader = DataLoader(DualInputDataset(q_train, a_train, g_train), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(DualInputDataset(q_test, a_test, g_test), batch_size=batch_size)

    model = MultiInputFusionModel(input_dim=1024, output_dim=128).to(device)
    criterion = nn.CosineEmbeddingLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    best_val_loss = float("inf")
    early_stop_counter = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for q, a, g in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            q, a, g = q.to(device), a.to(device), g.to(device)
            optimizer.zero_grad()
            out = model(q, a)
            out = F.normalize(out, dim=1)
            g = F.normalize(g, dim=1)
            target = torch.ones(out.size(0)).to(device)
            loss = criterion(out, g, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            total_loss += loss.item()
        scheduler.step()
        avg_train_loss = total_loss / len(train_loader)
        print(f"✅ Epoch {epoch+1} | Train Loss = {avg_train_loss:.4f}")

        # Validation step (train_loss used here as proxy)
        if avg_train_loss < best_val_loss:
            best_val_loss = avg_train_loss
            torch.save(model.state_dict(), output_model_path)
            print(f"💾 Saved best model to {output_model_path}")
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= early_stop_patience:
                print(f"🛑 Early stopping triggered at epoch {epoch+1}")
                break

    print("🔍 Evaluating on test set...")
    model.load_state_dict(torch.load(output_model_path))
    model.eval()
    test_loss = 0
    total_cos_sim = 0
    total_samples = 0

    with torch.no_grad():
        for q, a, g in test_loader:
            q, a, g = q.to(device), a.to(device), g.to(device)
            out = model(q, a)
            norm_out = F.normalize(out, dim=1)
            norm_g = F.normalize(g, dim=1)
            target = torch.ones(norm_out.size(0)).to(device)
            loss = criterion(norm_out, norm_g, target)
            test_loss += loss.item()
            cos_sim = (norm_out * norm_g).sum(dim=1).mean().item()
            total_cos_sim += cos_sim * q.size(0)
            total_samples += q.size(0)

    avg_test_loss = test_loss / len(test_loader)
    avg_cos_sim = total_cos_sim / total_samples
    print(f"🎯 Test Cosine Loss: {avg_test_loss:.4f}")
    print(f"📏 Test Cosine Similarity: {avg_cos_sim:.4f}")

    return model


# Main

In [7]:
train_mapping_model()

Using device: cuda
📥 Loading encoder: intfloat/e5-large-v2


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

Epoch 299: 100%|██████████| 32/32 [00:00<00:00, 85.32it/s]


✅ Epoch 299 | Train Loss = 0.0862


Epoch 300: 100%|██████████| 32/32 [00:00<00:00, 84.32it/s]
/tmp/ipykernel_19/873786439.py:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(

✅ Epoch 300 | Train Loss = 0.0833
🔍 Evaluating on test set...
🎯 Test Cosine Loss: 0.0483
📏 Test Cosine Similarity: 0.9507


MultiInputFusionModel(
  (q_mapper): MappingTransformer(
    (input_layer): Linear(in_features=1024, out_features=1024, bias=True)
    (layers): ModuleList(
      (0-5): 6 x Sequential(
        (0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (1): ReLU()
        (2): Dropout(p=0.3, inplace=False)
        (3): Linear(in_features=1024, out_features=1024, bias=True)
      )
    )
    (output_layer): Sequential(
      (0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (1): ReLU()
      (2): Dropout(p=0.3, inplace=False)
      (3): Linear(in_features=1024, out_features=128, bias=True)
    )
  )
  (a_mapper): MappingTransformer(
    (input_layer): Linear(in_features=1024, out_features=1024, bias=True)
    (layers): ModuleList(
      (0-5): 6 x Sequential(
        (0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (1): ReLU()
        (2): Dropout(p=0.3, inplace=False)
        (3): Linear(in_features=1024, out_features=1024, bias=True)
      )